# 1) Mount drive, unzip data, clone repo, install packages

## 1.1) Mount Drive and define paths
Run provided colab code to mount Google Drive. Then define dataset paths relative to mount point.

In [ ]:
!rm -rf /content/sample_data
!rm -rf /content/*.jpg
!rm -rf /content/*.png
!rm -rf /content/*.json

In [ ]:
# noinspection PyUnresolvedReferences
from google.colab import drive
mount_root_abs = '/content/drive'
drive.mount(mount_root_abs)
drive_root = f'{mount_root_abs}/MyDrive'

In [ ]:
import os
# DeepFashion In-shop Clothes Retrieval Benchmark (ICRB)
df_root_drive = f'{drive_root}/Datasets/DeepFashion'
assert os.path.exists(df_root_drive)
df_icrb_root_drive = f'{df_root_drive}/In-shop Clothes Retrieval Benchmark'
assert os.path.exists(df_icrb_root_drive)
df_icrb_img_zip_abs_drive = f'{df_icrb_root_drive}/Img.zip'

# If Img.zip is not present, we need to unzip .../Img/img_iuv.zip directory 
# from drive root and then run ICRBScraper.run() from /src/dataset/deep_fashion.
# For this nb, we skip this since it'll take an eternity to complete with 
# mounted Google Drive.
assert os.path.exists(df_icrb_img_zip_abs_drive), \
  'Please upload a processed zip (processing img.zip in colab will take' + \
  f' for AGES). \nTried: {df_icrb_img_zip_abs_drive}'

## 1.2) Unzip Img directory in Colab
By unzipping the Img_processed.zip in Colab before running our model we gain significant disk reading speedups.  
So, the first step is to unzip images directory, and then save the image directory before proceeding.

In [ ]:
df_icrb_root = df_icrb_root_drive.replace(drive_root, '/content/data')
df_icrb_img_root = f'{df_icrb_root}/Img'
if not os.path.exists(df_icrb_img_root):
  # Clear any previous attempts
  # ATTENTION: This will remove /contents/data/*. So, before running, please make 
  # sure no usable files will be deleted.
  !mkdir -p /content/data
  !rm -rf /content/data

  # Create output directory
  !mkdir -p "$df_icrb_root"

  # Transfer Img.zip from Google Drive to Colab
  df_icrb_img_zip_abs = f'{df_icrb_root}/{os.path.basename(df_icrb_img_zip_abs_drive)}'
  if not os.path.exists(df_icrb_img_zip_abs):
    !cp "$df_icrb_img_zip_abs_drive" "$df_icrb_root"
  # Unzip it in Colab
  !unzip -q "$df_icrb_img_zip_abs" -d "$df_icrb_root"
  # Handle newly-created image directory
  assert os.path.exists(df_icrb_img_root), f'df_icrb_img_root: {df_icrb_img_root}'
  assert not os.path.exists(f'{df_icrb_img_root}/Img')
  assert not os.path.exists(f'{df_icrb_img_root}/img')
  !rm -f "$df_icrb_img_zip_abs"
  assert not os.path.exists(df_icrb_img_zip_abs)

## 1.3) Clone github repo
Clone achariso/gans-thesis repo into /content/code
 using git clone.  
 For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [ ]:
repo_root = '/content/code/gans-thesis'
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
  # Check that ssh keys exist
  assert os.path.exists(f'{drive_root}/GitHub Keys')
  id_rsa_abs_drive = f'{drive_root}/GitHub Keys/id_rsa'
  id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
  assert os.path.exists(id_rsa_abs_drive)
  assert os.path.exists(id_rsa_pub_abs_drive)
  # On first run: Add ssh key in repo
  # !cat "$id_rsa_pub_abs_drive", copy & paste in repo's Deploy Keys
  # Transfer config file
  ssh_config_abs_drive = f'{drive_root}/GitHub Keys/config'
  assert os.path.exists(ssh_config_abs_drive)
  !mkdir -p ~/.ssh
  !cp -f "$ssh_config_abs_drive" ~/.ssh/
  # # Add github.com to known hosts
  !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
  # Test: !ssh -T git@github.com

  # Remove any previous attempts
  !mkdir -p "$repo_root"
  !rm -rf "$repo_root"
  !mkdir -p "$repo_root"
  # Clone repo
  !git clone git@github.com:achariso/gans-thesis.git "$repo_root"
  src_root = f'{repo_root}/src'
  !rm -rf "$repo_root"/report

## 1.4) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [ ]:
%cd $repo_root
!pip install -r requirements.txt

In [ ]:
import torch
assert torch.cuda.is_available()

## 1.6) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [ ]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/env/python:$content_root_abs:$src_root_abs
%env PATH="$PATH:$content_root_abs:$src_root_abs

# 2) Test code
Test that pulled code is running by running its tests.

In [ ]:
import matplotlib.pyplot as plt

# Fix: Adjust figure size for better plotting in Colab
plt.rcParams["figure.figsize"] = (20, 20)

In [ ]:
%run src/main.py

In [ ]:
test_root = f'{repo_root}/tests'
!python -m unittest discover -s "$test_root" -t "$test_root"